In [140]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor
from feature import Feature

In [19]:
import json
from pathlib import Path
def read_conv(path:str, datalist:list):
    convs = []
    for p in datalist:
        datapath = Path(path + p + '/')
        for file_ in datapath.glob("*.json"):
            conv = []
            with open(file_, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    sp = t["speaker"]
                    utt = t["utterance"]
                    errors = t["error_category"]
                    type_ = t["type"]
                    one = Utterance(did, sp, utt, errors, type_)
                    conv.append(one)
            convs.append(conv)
    return convs      

In [20]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']


In [21]:
convs = read_conv(path, datalist)

In [113]:
# 前向き発話以外は後ろ向きと考える
def extract_response(convs, type_):
    type_utt = []
    forward = []
    backward = []
    plain = []
    for conv in convs:
        for i, ut in enumerate(conv):
            if i == len(conv)-1:
                continue
            # システムの質問に対する応答
            # if ut.is_system() and ut.is_type_included(type_) and not ut.is_exist_error():
            #     if not ut.is_utt_level_error():
            #         if conv[i+1].is_exist_type():
            #             print("後ろ+前", conv[i+1], conv[i+1].did)
            #         else:
            #             print("後ろ", conv[i+1])

            # システムの場合\
            
            if ut.is_type_included("挨拶"):
                continue
            utt = clean_text( ut.utt )
            utt = utt.replace("\u3000", " ")
            utt = utt.replace("？？", "？")
            if ut.is_system():
                # 発話レベルのエラーではなく
                if not ut.is_utt_level_error():
                    # 前向きである
                    if not ut.is_exist_error():
                        if ut.is_exist_type():
                        # # 後ろ+前ではない
                        # if not conv[i-1].is_exist_type():
                            forward.append(utt)
                        else:
                            plain.append(utt)
                            pass
                        
            # ユーザ発話
            else:
                # 前向き
                if ut.is_exist_type():
                    if conv[i-1].is_exist_type():
                        forward.append(utt)
                # ユーザで，後ろ向き
                else:
                    # 直前の発話は
                    if conv[i-1].is_exist_type():
                        backward.append(utt)
                    else:
                        backward.append(utt)

            
            # システムの前向き 
            # if ut.is_system() and not ut.is_utt_level_error():
            #     if ut.is_exist_type():
            #         print("前向き", ut)
            #         # 後ろ+前
            #         if conv[i+1].is_exist_type():
            #             print("後ろ+前", conv[i+1], conv[i+1].did)
            #         # 後ろ
            #         else:
            #             print("後ろ", conv[i+1])
            #         print()
    
            #     else:
            #         if not ut.is_exist_error():
            #             print("not type", conv[i+1])
            #             print()
                    
    return forward, backward, plain

In [114]:
forward, backward, plain = extract_response(convs, "提案")

In [143]:
X = (forward+backward)
y = np.concatenate( [ np.zeros(len(forward)), np.zeros(len(backward))+1] )

In [144]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [145]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [146]:
F = Feature()
F.set_preprocessor(Preprocessor())
F.make_features(X_train_str)


300


In [153]:
F_path = "../X_y_data/response/"
F_name = "forback1.pickle"
featureM = DataManager(F_path)
featureM.save_data(F_name, F)

success save : ../X_y_data/response/forback1.pickle


In [147]:
print(F.feature_num)
X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

20642


In [148]:
lr = LogisticRegression(solver='sag', max_iter=10000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=10000, solver='sag')

In [155]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [156]:
# y_pred = lr.predict(X_test)
y_pred = clf.predict(X_test)

In [150]:

for y_p, x_s in zip(y_pred[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p, x_s))

1.0 : それは私の質問の答えではないですよ？
1.0 : 倖田來未さんはアナウンサーではないですよ！
1.0 : 海はいいですよね。
1.0 : ヒルトンですね
1.0 : そうかも知れませんね。
1.0 : そうだね。適切に水分をと取らないとね。
1.0 : お遍路さんは、歩くのが辛そうですね。
1.0 : 知らなかった。お詳しいんですね。
1.0 : 犬です
1.0 : ああ。周りの女子の話か。そう？
1.0 : 役立つというか達成感がいいわ
0.0 : では、あなたの趣味は何ですか？
1.0 : それ知ってます、私も見に行きました。
1.0 : こんにちは。すみません、二人とも知りません。
1.0 : 危険ですよ。水分補給をしてくださいね。
1.0 : そうかもしれないですね。今日は暑いですね。
1.0 : 関西方面へ行かれることがありましたら。ぜひ関西のメロンパンの皮焼いちゃいましたを御覧になってみてください。
1.0 : 他にもシューマイ等色々名物がありますよ。
1.0 : 楽しいですね。
1.0 : 物知りですね
1.0 : 涼しくなってきたら、一緒に山へ行きたいですね。
1.0 : 維持費が結構かかりますよ。
1.0 : たまにだね
1.0 : 行くかどうかどうかは時と場合によりますね
1.0 : こんばんは、夏といえば海ですね。
1.0 : ニュージーランドが楽しいと思います。
1.0 : こんにちは。怖いですよね熱中症。毎日蒸し暑いですからね。
1.0 : どんな御菓子でも、薄塩味は外しませんよね
1.0 : 恋愛こそケースバイケースな判断力が必要となりますしね
1.0 : そうなんですね。


In [157]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[ 47  14]
 [ 10 403]]
accuracy =  0.9493670886075949
precision =  0.9664268585131894
recall =  0.9757869249394673
f1 score =  0.9710843373493975


In [158]:
model_path = "../models/response/"
model_name = "forback_clf.pickle"
modelM = DataManager(model_path)
print(model_name)
modelM.save_data(model_name, lr)

forback_clf.pickle
success save : ../models/response/forback_clf.pickle
